# **Импорт библиотек**

In [1]:
import pandas as pd
import dill
import os
from functools import partial

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# **Загрузка и подготовка данных**

In [2]:
data_path = 'Сводная 2.xlsx'

In [3]:
try:
    if data_path.endswith('.xlsx'):
        data = pd.read_excel(data_path)

    elif data_path.endswith('.csv'):
        data = pd.read_csv(data_path)

except Exception as e:
    raise ValueError(f"Ошибка при загрузке данных: {e}")

In [4]:
def preprocess_df(df: pd.DataFrame, threshold_percent: float = 99, del_first_row: bool = True) -> pd.DataFrame:
    """
    Очищает DataFrame, удаляя столбцы с пропусками выше порога и (опционально) первую строку.

    :param df: Входной DataFrame
    :param threshold_percent: Порог удаления столбцов (в процентах пропусков)
    :param del_first_row: Флаг удаления первой строки
    :return: Очищенный DataFrame
    """
    if not (0 <= threshold_percent <= 100):
        raise ValueError("Порог должен быть в диапазоне от 0 до 100.")
    
    df = df.iloc[1:].reset_index(drop=True) if del_first_row else df.copy()
    
    threshold = threshold_percent / 100
    columns_to_keep = df.columns[df.isnull().mean() <= threshold]
    
    return df[columns_to_keep].fillna("")

data = preprocess_df(data)

# **Обучение модели**

### Определение колонок

In [6]:
# ---------------------------------------------------------------------------------------
# Автоматическое определение категориальных колонок на основе кол-ва уникальных значений
# ---------------------------------------------------------------------------------------

TARGET_COL = "Группа МТР (ред.)"
CATEGORICAL_THRESHOLD = 150  # Порог уникальных значений для категориальных данных

# Разделение колонок на категориальные и текстовые
categorical_cols = [
    col for col in data.columns 
    if col != TARGET_COL and data[col].nunique() <= CATEGORICAL_THRESHOLD
]

text_cols = [
    col for col in data.columns 
    if col != TARGET_COL and data[col].nunique() > CATEGORICAL_THRESHOLD
]

print("Категориальные столбцы:", categorical_cols)
print("Текстовые столбцы:", text_cols)

Категориальные столбцы: ['Марка РД', 'Ед. изм.']
Текстовые столбцы: ['Чертёж', 'Наименование МТР', 'Тип, марка ГОСТ МТР']


In [5]:
# ---------------------------
# Ручное определение колонок
# ---------------------------

TARGET_COL = "Группа МТР (ред.)"
categorical_cols = ['Чертёж', 'Тип, марка ГОСТ МТР', 'Марка РД']
text_cols = ['Наименование МТР', 'Ед. изм.']

### Обучение модели

In [6]:
# Класс для гибкости пайплайна. Если на вход подаются данные без указанных столбцов, то создаём их с пустыми значениями.
class SafeReindexer(BaseEstimator, TransformerMixin):
    def __init__(self, required_cols, fill_value=None):
        self.required_cols = required_cols
        self.fill_value = fill_value

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Преобразуем в DataFrame, чтобы удобно было добавлять столбцы
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)

        # reindex добавит недостающие столбцы, заполнит их fill_value
        # а лишние столбцы сохранит (можно указать, что делать с ними).
        X = X.reindex(columns=self.required_cols, fill_value=self.fill_value)

        return X

# --- Пайплайн для "категориальных" колонок ---
cat_pipeline = Pipeline([
    # Переводим в строки все значения
    ('astype_str', FunctionTransformer(lambda x: x.astype(str), validate=False)),
    # OneHotEncoder для кодирования категориальных признаков
    ("onehot", OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

# --- Пайплайн для "текстовых" колонок ---
# Объединяем все колонки в один текст и подаём в TfidfVectorizer
text_pipeline = Pipeline([
    (
        "to_df",
        FunctionTransformer(
            partial(lambda arr, cols: pd.DataFrame(arr, columns=cols), cols=text_cols),
            validate=False
        )
    ),
    (
        "combine_text",
        FunctionTransformer(
            lambda df: df.apply(lambda row: " ".join(row.astype(str)), axis=1),
            validate=False
        )
    ),
    ("tfidf", TfidfVectorizer())
])

# --- ColumnTransformer для параллельной обработки ---
preprocessor = ColumnTransformer([
    ("cat", cat_pipeline, categorical_cols),
    ("text", text_pipeline, text_cols)
])

# Итоговый пайплайн
pipeline = Pipeline([
    ("safe_reindex", SafeReindexer(required_cols=(categorical_cols + text_cols), fill_value="")),
    ("preprocessing", preprocessor),
    ("clf", RandomForestClassifier(random_state=42))
])

# Разделяем на признаки и целевую переменную
X = data.drop(columns=[TARGET_COL])
y = data[TARGET_COL]

# Тренировочная и тестовая выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

# Обучаем
pipeline.fit(X_train, y_train)

# Предсказываем
y_pred = pipeline.predict(X_test)

# Оцениваем результаты
print(classification_report(y_test, y_pred))

                                                         precision    recall  f1-score   support

                                    Ёмкости, резервуары       0.00      0.00      0.00         2
                                  Арматура строительная       1.00      0.98      0.99       183
                                                Асфальт       1.00      1.00      1.00         3
                                                  Бетон       0.99      0.97      0.98        70
                             Бытовая техника, оргтехика       1.00      0.50      0.67         8
         Вагончики, контейнеры, блочно-модульные здания       0.67      1.00      0.80         2
                                            Воздуховоды       0.98      1.00      0.99        63
                                    Вспомогательные МТР       1.00      0.80      0.89         5
                                           Газоходы (т)       1.00      1.00      1.00        18
                             

g:\m108\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
g:\m108\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
g:\m108\venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Сохранение модели

In [9]:
# Сохранение модели
save_path = "models/model_mtp_group.pkl"

# Создание папки, если её нет
os.makedirs(os.path.dirname(save_path), exist_ok=True)

with open(save_path, "wb") as f:
    dill.dump(pipeline, f)